#### Imports/setup

In [1]:
import ee
import pandas as pd
import geopandas as gpd
import json
import geemap
import time
import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import get_ceo_url, whisp_stats_shapefile_and_ceo_url
from parameters.config_output_naming import geo_id_column, out_file_wide, out_shapefile
from modules.agstack_to_gee import start_agstack_session, get_agstack_token, geometry_to_geo_id, feature_to_geo_id,geo_id_or_ids_to_feature_collection, register_feature_and_set_geo_id, register_fc_and_set_geo_id, shapefile_to_ee_feature_collection 
from parameters.config_asr_url_info import asset_registry_base,user_registry_base
from parameters.config_asr_credentials import email,password

def whisp_alt_stats_as_df (roi): 
    %store roi
    %run alt_whisp_stats.ipynb   
    return df_out

# Compute centroids of each polygon
def getCentroid(feature):
    keepProperties = [geo_id_column];
    # Get the centroid of the feature's geometry.
    centroid = feature.geometry().centroid(1);
    #Return a new Feature, copying properties from the old Feature.
    return ee.Feature(centroid).copyProperties(feature, keepProperties);
    
#set up asset registry
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)


Earth Engine has been initialized with the specified credentials.


#### Inputs : with GEOIDS

##### Input: list of Geo ids

In [11]:
example_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids
#all_geo_ids= example_ids

In [12]:
#make into a feature collection
roi_geoid = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

In [40]:
geo_id_column

'Geo_id'

#### Inputs : without GEOIDS

##### Input: GEE asset (feature collection) 

In [2]:
example_plot_feature_col_asset = "projects/fdap-remi/assets/seg_civ_scoopac"

# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0025

In [3]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)#.limit(10)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

fc = fc#_sample

In [ ]:
#run for feature collection (NB needs speeding up..get geo ids then use map over property setting)
roi_asset = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True
)

using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  11ac3c0b57353f173747a0275cee3a00187e0a73458f3a088082aa3d2ed7d641
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  d091a16acf45e2a855ae6be628a1d3d02724df7bc36ff33312114b94b066df67
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  5086df340ad747fd4684d7fd406caa3536f576add47c35117804067ddfceda4c
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  6057cc646a1f0355ceb7513a828981be4e7384b87ecabb5f23983ce726741f50
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  9c0a96fe0c7449b8009a618c4720002a88fed9db9dc4988c0223638fbee8c717
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id

In [72]:
roi_asset

##### Input: polygon feature collection

In [ ]:
geom_1 =ee.Geometry.Polygon(
        [[[102.19867776108482, 3.0835809888660344],
          [102.1988172359536, 3.082670357801012],
          [102.20001886559227, 3.082078035157836],
          [102.2013385124276, 3.081745922272887],
          [102.2014355657636, 3.0825815609469625],
          [102.20011591892828, 3.0836528916202233]]],None, False);

geom_2 = ee.Geometry.Polygon(
        [[[102.18777620396249, 3.079459767638651],
          [102.18831264576546, 3.0785384196951506],
          [102.1895786484205, 3.0792026473624907],
          [102.18899929127328, 3.08005971470826]]],None, False);

feat_1 = ee.Feature(geom_1)
feat_2 = ee.Feature(geom_2)

In [ ]:
fc = ee.FeatureCollection([feat_1,feat_2])

In [ ]:
#run for feature collection (NB needs speeding up..get geo ids then use map over property setting)
roi_asset = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=False
)

##### Input: point feature collection

In [ ]:
#get test points from polygon feature collection (NB functionality for registering points in asset registry coming soon)
fc.map(getCentroid)

In [ ]:
roi_poly = ee.FeatureCollection(fc.map(getCentroid))

##### Input: shapefile

In [ ]:
# example_shapefile = /path_to/my_shapefile
example_shapefile = out_shapefile

In [ ]:
roi_shapefile = shapefile_to_ee_feature_collection(example_shapefile)

##### Input: drawn feature

In [ ]:
zoom_level =15

Map = geemap.Map(center=[6,-4], zoom=15)

# Map.add_basemap("Esri.WorldImagery")# optional basemaps from geemap

sc = Sidecar(title='draw polygon')

with sc:display(Map)
print("Draw shape on map before running next cell")

In [ ]:
#NB User input needed: draw boundary (under 400ha) on the map before running this cell
drawn_feature = (Map.draw_last_feature)

In [ ]:
#register geo id and add to feature
feature_w_geo_id = register_feature_and_set_geo_id(
    drawn_feature,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True)

In [ ]:
roi_drawn = feature_w_geo_id

#### Get plot boundaries

In [66]:
#roi = roi_geoid
roi = roi_asset
#roi = roi_shapefile
#roi = roi_polygon
#roi = roi_points
#roi = roi_drawn

### Whisp it

In [67]:
df = whisp_alt_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
...ee stats processing time: 1.5112190246582031 seconds
...ee to df processing time: 2.933791160583496 seconds
...table tidying time: 0.008920669555664062 seconds.
 Processing finished


### Display it

In [68]:
df

,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,4ca4a0d1357ec00cd306694513ceb3df1dc8556848b1f6...,2.6,CIV,19,0,0,100,100,19,92,0,-,0,0,0,0,-
2,bea47159f76bfcd2629fb750899c27c446528aa2ab635b...,2.9,CIV,0,0,0,47,41,0,68,0,-,0,0,0,0,-
3,0751b8b554c9f84e53c022747fd5937a82248967a37963...,3.1,CIV,17,0,0,91,92,12,30,11,-,0,0,0,0,-
4,0fd4b07d67052f96ca232d201d9f62c26e0661bb9e84fc...,4.4,CIV,36,0,0,93,100,29,54,0,-,0,0,0,0,-
5,388b711c2433d2fb7426314fc51529d0888f05400e4fea...,4.5,CIV,1,0,0,87,94,2,61,0,-,0,0,0,0,-
6,03b42354602fc1f162b8b57bfaa7b5b99ea11614e9ba6f...,6.5,CIV,44,0,1,99,79,94,98,27,-,0,0,0,91,-
7,a387f76ecc48ee4e12dac9d73aab25906491bbcedf9538...,7.5,CIV,30,0,1,96,66,52,65,28,-,0,0,0,58,-
8,5709434a296be348d18530ed0ce296dff54a3cf5b6289e...,7.7,CIV,0,0,0,7,100,0,2,0,-,0,0,0,0,-
9,a20c3c9b7aec89873d1caa958410f841bf7b4a205d49de...,8.2,CIV,1,0,0,23,92,7,6,1,-,0,0,0,0,-


#### Export it to CSV file

In [ ]:
#export output csv
df.to_csv(path_or_buf=out_file_wide, header=True)

print ("output csv: ", out_file_wide)

#### Export it to CEO link

Temporary login details available for demo purposes here: modules/config_ceo.py

In [ ]:
ceo_url = whisp_stats_shapefile_and_ceo_url(roi, df, plot_id_column, out_shapefile)

print('CEO URL:', ceo_url) 